# 在本次任务中，我们将手动实现 Softmax 回归模型，并使用 PyTorch 的 Tensor 和 NumPy 库在 Fashion-MNIST 数据集上进行训练和测试。我们还会从零实现交叉熵损失函数，并计算训练集和测试集的准确率。

# 数据集加载与处理
首先，需要下载并加载 Fashion-MNIST 数据集，使用 PyTorch 提供的 torchvision 来处理该数据集。对数据进行标准化处理。

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 数据集加载与预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 加载训练和测试集
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Softmax 函数与模型定义
Softmax 函数是对每个类别的得分进行归一化，使得输出是一个概率分布。将从零实现 Softmax 回归模型。

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义 Softmax 回归模型
class SoftmaxRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)  # 定义线性层

    def forward(self, x):
        logits = self.linear(x)  # 线性层的输出
        softmax_output = F.softmax(logits, dim=1)  # 应用 softmax 归一化
        return softmax_output

# 实现交叉熵损失函数
交叉熵损失函数可以通过以下公式实现。需要确保输出不会为 0，否则会导致 log(0) 的情况。

In [38]:
# 实现交叉熵损失函数
def cross_entropy_loss(y_pred, y_true):
    epsilon = 1e-12  # 防止 log(0)
    y_pred = torch.clamp(y_pred, epsilon, 1. - epsilon)  # 限制 y_pred 在 (epsilon, 1-epsilon) 范围内
    y_true_one_hot = torch.zeros(y_pred.shape)
    y_true_one_hot[range(len(y_true)), y_true] = 1
    loss = -torch.sum(y_true_one_hot * torch.log(y_pred)) / y_pred.shape[0]
    return loss

# 模型训练
接下来，实现训练函数。在每个 epoch 中，通过前向传播计算损失，反向传播更新权重。

In [39]:
# 训练模型
def train(model, train_loader, learning_rate, num_epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)  # 调整学习率/使用 model.parameters() 自动优化所有参数
    
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        
        for images, labels in train_loader:
            images = images.view(-1, 28*28)  # 将图像展平成一维向量
            
            # 前向传播
            outputs = model.forward(images)
            
            # 计算损失
            loss = cross_entropy_loss(outputs, labels)
            total_loss += loss.item()
            
            # 反向传播与优化
            optimizer.zero_grad()  # 梯度清零
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            
            # 计算准确率
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == labels).sum().item()
        
        accuracy = total_correct / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}')


In [40]:
# 初始化模型并开始训练
model = SoftmaxRegression(28*28, 10)  # 输入大小为 28x28，类别数量为 10
train(model, train_loader, learning_rate=0.1, num_epochs=100)

Epoch [1/100], Loss: 2.0026, Accuracy: 0.3347
Epoch [2/100], Loss: 1.5340, Accuracy: 0.5799
Epoch [3/100], Loss: 1.3080, Accuracy: 0.6534
Epoch [4/100], Loss: 1.1711, Accuracy: 0.6865
Epoch [5/100], Loss: 1.0787, Accuracy: 0.7035
Epoch [6/100], Loss: 1.0119, Accuracy: 0.7139
Epoch [7/100], Loss: 0.9610, Accuracy: 0.7218
Epoch [8/100], Loss: 0.9209, Accuracy: 0.7289
Epoch [9/100], Loss: 0.8883, Accuracy: 0.7337
Epoch [10/100], Loss: 0.8611, Accuracy: 0.7380
Epoch [11/100], Loss: 0.8383, Accuracy: 0.7419
Epoch [12/100], Loss: 0.8188, Accuracy: 0.7453
Epoch [13/100], Loss: 0.8014, Accuracy: 0.7486
Epoch [14/100], Loss: 0.7861, Accuracy: 0.7516
Epoch [15/100], Loss: 0.7726, Accuracy: 0.7544
Epoch [16/100], Loss: 0.7606, Accuracy: 0.7567
Epoch [17/100], Loss: 0.7496, Accuracy: 0.7594
Epoch [18/100], Loss: 0.7395, Accuracy: 0.7619
Epoch [19/100], Loss: 0.7303, Accuracy: 0.7647
Epoch [20/100], Loss: 0.7220, Accuracy: 0.7663
Epoch [21/100], Loss: 0.7140, Accuracy: 0.7683
Epoch [22/100], Loss: 

# 测试模型
在训练完成后，需要在测试集上评估模型的表现，计算损失和准确率。

In [41]:
# 模型测试
def test(model, test_loader):
    total_correct = 0
    total_loss = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 28*28)  # 将图像展平成一维向量
            
            # 前向传播
            outputs = model.forward(images)
            
            # 计算损失
            loss = cross_entropy_loss(outputs, labels)
            total_loss += loss.item()
            
            # 计算准确率
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == labels).sum().item()
    
    accuracy = total_correct / len(test_loader.dataset)
    print(f'Test Loss: {total_loss/len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}')

# 测试模型
test(model, test_loader)

Test Loss: 0.5606, Test Accuracy: 0.8051


# 分析结果
+ Loss: 训练过程中的损失会逐渐减小，表明模型在逐渐拟合训练数据。
+ Accuracy: 训练和测试的准确率会逐渐提高，并最终达到一定的稳定值，表明模型能正确分类大部分样本。

# 总结
通过从零实现 Softmax 回归模型，并手动实现交叉熵损失函数，我们成功在 Fashion-MNIST 数据集上训练了模型，并测试了其表现。通过不断优化模型参数，模型的损失逐渐降低，准确率逐渐提高，最终能较好地对图像数据进行分类。